# Exercise 2 - DataLoader
In the previous notebook you have implemented a dataset that we can now use to access our data. However, in machine learning, we often need to perform a few additional data preparation steps before we can start training models.

An important additional class for data preparation is the **DataLoader**. By wrapping a dataset in a dataloader, we will be able to load small subsets of the dataset at a time, instead of having to load each sample separately. In machine learning, the small subsets are referred to as **mini-batches**, which will play an important role later in the lecture.

In this notebook, you will implement your own dataloader, which you can then use to load mini-batches from the dataset you implemented previously.


## Exercise Group Information
- Group Number: Group 11
- Group Members: Malik Abdoul Hamidou, Maurice Todtenbier, Niclas Gambal, Yannek Wunderlich

## (Optional) Mount folder in Colab

Uncomment thefollowing cell to mount your gdrive if you are using the notebook in google colab:

In [1]:
# Use the following lines if you want to use Google Colab
# We presume you created a folder "i2dl" within your main drive folder, and put the exercise there.
# NOTE: terminate all other colab sessions that use GPU!
# NOTE 2: Make sure the correct exercise folder (e.g exercise_03) is given.
# NOTE 3: To use, remove the triple " from the beginning and end of this cell.

"""
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/i2dl/exercise_03'

# This will mount your google drive uWnder 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))

"""

"\nfrom google.colab import drive\nimport os\n\ngdrive_path='/content/gdrive/MyDrive/i2dl/exercise_03'\n\n# This will mount your google drive uWnder 'MyDrive'\ndrive.mount('/content/gdrive', force_remount=True)\n# In order to access the files in this notebook we have to navigate to the correct folder\nos.chdir(gdrive_path)\n# Check manually if all files are present\nprint(sorted(os.listdir()))\n\n"

## Imports

First, you need to import libraries and code, as always.

In [2]:
!uv sync --frozen

Using CPython 3.12.12
Creating virtual environment at: .venv
Installed 38 packages in 2.46s
 + asttokens==3.0.0
 + colorama==0.4.6
 + comm==0.2.3
 + contourpy==1.3.3
 + cycler==0.12.1
 + debugpy==1.8.17
 + decorator==5.2.1
 + executing==2.2.1
 + fonttools==4.60.1
 + ipykernel==7.0.1
 + ipynb==0.5.1
 + ipython==9.6.0
 + ipython-pygments-lexers==1.1.1
 + jedi==0.19.2
 + jupyter-client==8.6.3
 + jupyter-core==5.9.1
 + kiwisolver==1.4.9
 + matplotlib==3.10.7
 + matplotlib-inline==0.1.7
 + nest-asyncio==1.6.0
 + numpy==2.3.4
 + packaging==25.0
 + parso==0.8.5
 + pillow==12.0.0
 + platformdirs==4.5.0
 + prompt-toolkit==3.0.52
 + psutil==7.1.1
 + pure-eval==0.2.3
 + pygments==2.19.2
 + pyparsing==3.2.5
 + python-dateutil==2.9.0.post0
 + pyzmq==27.1.0
 + six==1.17.0
 + stack-data==0.6.3
 + tornado==6.5.2
 + tqdm==4.67.1
 + traitlets==5.14.3
 + wcwidth==0.2.14


In [4]:
import numpy as np
import os
from exercise_code.data import DataLoader, DummyDataset
from exercise_code.tests import (
    test_dataloader, 
    test_dataloader_len,
    test_dataloader_iter,
    save_pickle, 
    load_pickle
)

%load_ext autoreload
%autoreload 2

os.environ['KMP_DUPLICATE_LIB_OK']='True' # To prevent the kernel from dying.

## 1. Iterating over a Dataset
Throughout this notebook a dummy dataset will be used that contains all even numbers from 2 to 100. Similar to the dataset you have implemented before, the dummy dataset has a `__len__()` method that allows us to call `len(dataset)`, as well as a `__getitem__()` method, which allows you to call `dataset[i]` and returns a dict `{"data": val}` where `val` is the i-th even number. If you would like to see the code, have a look at `DummyDataset` in `exercise_code/data/base_dataset.py`.

Let's start by defining the dataset, and calling its methods to get a better feel for it.

<div class="alert alert-info">
    <h3>Task: Implement (1 Point)</h3>
    <p>Print the length, first and last element of the dataset.</p>
</div>

In [6]:
from exercise_code.data.base_dataset import DummyDataset

dataset = DummyDataset(
    root=None,
    divisor=2,
    limit=100
)
print(
    "Dataset Length:\t", len(dataset), # TODO: Print the length of the dataset
    "\nFirst Element:\t", dataset[0], # TODO: Print the first element of the dataset
    "\nLast Element:\t", dataset[len(dataset) - 1], # TODO: Print the last element of the dataset
)

Dataset Length:	 50 
First Element:	 {'data': 2} 
Last Element:	 {'data': 100}


In the following, you will write some code to iterate over the dataset in mini-batches, similarly to what a dataloader is supposed to do. The number of samples to load per mini-batch is called a **batch size**. For the rest of this notebook, the batch size is 3.

In [8]:
batch_size = 3

Let us now define a simple function that iterates over the dataset and groups samples into mini-batches:

In [10]:
def build_batches(dataset, batch_size):
    batches = []  # list of all mini-batches
    batch = []  # current mini-batch
    for i in range(len(dataset)):
        batch.append(dataset[i])
        if len(batch) == batch_size:  # if the current mini-batch is full,
            batches.append(batch)  # add it to the list of mini-batches,
            batch = []  # and start a new mini-batch
    return batches

batches = build_batches(
    dataset=dataset,
    batch_size=batch_size
)

Let's have a look at the mini-batches:

In [12]:
def print_batches(batches):  
    for i, batch in enumerate(batches):
        print("mini-batch %d:" % i, str(batch))

print_batches(batches)

mini-batch 0: [{'data': 2}, {'data': 4}, {'data': 6}]
mini-batch 1: [{'data': 8}, {'data': 10}, {'data': 12}]
mini-batch 2: [{'data': 14}, {'data': 16}, {'data': 18}]
mini-batch 3: [{'data': 20}, {'data': 22}, {'data': 24}]
mini-batch 4: [{'data': 26}, {'data': 28}, {'data': 30}]
mini-batch 5: [{'data': 32}, {'data': 34}, {'data': 36}]
mini-batch 6: [{'data': 38}, {'data': 40}, {'data': 42}]
mini-batch 7: [{'data': 44}, {'data': 46}, {'data': 48}]
mini-batch 8: [{'data': 50}, {'data': 52}, {'data': 54}]
mini-batch 9: [{'data': 56}, {'data': 58}, {'data': 60}]
mini-batch 10: [{'data': 62}, {'data': 64}, {'data': 66}]
mini-batch 11: [{'data': 68}, {'data': 70}, {'data': 72}]
mini-batch 12: [{'data': 74}, {'data': 76}, {'data': 78}]
mini-batch 13: [{'data': 80}, {'data': 82}, {'data': 84}]
mini-batch 14: [{'data': 86}, {'data': 88}, {'data': 90}]
mini-batch 15: [{'data': 92}, {'data': 94}, {'data': 96}]


As you can see, the iteration works, but the output is not very pretty. Let us now write a simple function that combines the dictionaries of all samples in a mini-batch.

In [14]:
def combine_batch_dicts(batch):
    batch_dict = {}
    for data_dict in batch:
        for key, value in data_dict.items():
            if key not in batch_dict:
                batch_dict[key] = []
            batch_dict[key].append(value)
    return batch_dict

combined_batches = [combine_batch_dicts(batch) for batch in batches]
print_batches(combined_batches)

mini-batch 0: {'data': [2, 4, 6]}
mini-batch 1: {'data': [8, 10, 12]}
mini-batch 2: {'data': [14, 16, 18]}
mini-batch 3: {'data': [20, 22, 24]}
mini-batch 4: {'data': [26, 28, 30]}
mini-batch 5: {'data': [32, 34, 36]}
mini-batch 6: {'data': [38, 40, 42]}
mini-batch 7: {'data': [44, 46, 48]}
mini-batch 8: {'data': [50, 52, 54]}
mini-batch 9: {'data': [56, 58, 60]}
mini-batch 10: {'data': [62, 64, 66]}
mini-batch 11: {'data': [68, 70, 72]}
mini-batch 12: {'data': [74, 76, 78]}
mini-batch 13: {'data': [80, 82, 84]}
mini-batch 14: {'data': [86, 88, 90]}
mini-batch 15: {'data': [92, 94, 96]}


This looks much more organized.

To perform operations more efficiently later, we would also like the values of the mini-batches to be contained in a numpy array instead of a simple list. Let's briefly write a function for that:

In [16]:
def batch_to_numpy(batch):
    numpy_batch = {}
    for key, value in batch.items():
        numpy_batch[key] = np.array(value)
    return numpy_batch

numpy_batches = [batch_to_numpy(batch) for batch in combined_batches]
print_batches(numpy_batches)

mini-batch 0: {'data': array([2, 4, 6])}
mini-batch 1: {'data': array([ 8, 10, 12])}
mini-batch 2: {'data': array([14, 16, 18])}
mini-batch 3: {'data': array([20, 22, 24])}
mini-batch 4: {'data': array([26, 28, 30])}
mini-batch 5: {'data': array([32, 34, 36])}
mini-batch 6: {'data': array([38, 40, 42])}
mini-batch 7: {'data': array([44, 46, 48])}
mini-batch 8: {'data': array([50, 52, 54])}
mini-batch 9: {'data': array([56, 58, 60])}
mini-batch 10: {'data': array([62, 64, 66])}
mini-batch 11: {'data': array([68, 70, 72])}
mini-batch 12: {'data': array([74, 76, 78])}
mini-batch 13: {'data': array([80, 82, 84])}
mini-batch 14: {'data': array([86, 88, 90])}
mini-batch 15: {'data': array([92, 94, 96])}


Lastly, we would like to make the loading a bit more memory efficient. Instead of loading the entire dataset into memory at once, let us only load samples when they are needed. This can also be done by building a Python generator, using the `yield` keyword.

Here are a few tutorials that might be helpful for learning about Python generators and iterators:

1. [“Python Iterators and Generators Tutorial”](https://www.datacamp.com/tutorial/python-iterators-generators-tutorial) by DataCamp
2. [“How to Use Generators and yield in Python”](https://realpython.com/introduction-to-python-generators/) by Real Python 
3. [“Python Generators (With Examples)”](https://www.programiz.com/python-programming/generator) by Programiz 

In [18]:
def build_batch_iterator(dataset, batch_size, shuffle=True):
    if shuffle:
        index_iterator = iter(np.random.permutation(len(dataset)))  # define indices as iterator
    else:
        index_iterator = iter(range(len(dataset)))  # define indices as iterator

    batch = []
    for index in index_iterator:  # iterate over indices using the iterator
        batch.append(dataset[index])
        if len(batch) == batch_size:
            yield batch  # use yield keyword to define a iterable generator
            batch = []
            
batch_iterator = build_batch_iterator(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=False
)
batches = []
for batch in batch_iterator:
    batches.append(batch)

print_batches(
    [batch_to_numpy(combine_batch_dicts(batch)) for batch in batches]
)

mini-batch 0: {'data': array([2, 4, 6])}
mini-batch 1: {'data': array([ 8, 10, 12])}
mini-batch 2: {'data': array([14, 16, 18])}
mini-batch 3: {'data': array([20, 22, 24])}
mini-batch 4: {'data': array([26, 28, 30])}
mini-batch 5: {'data': array([32, 34, 36])}
mini-batch 6: {'data': array([38, 40, 42])}
mini-batch 7: {'data': array([44, 46, 48])}
mini-batch 8: {'data': array([50, 52, 54])}
mini-batch 9: {'data': array([56, 58, 60])}
mini-batch 10: {'data': array([62, 64, 66])}
mini-batch 11: {'data': array([68, 70, 72])}
mini-batch 12: {'data': array([74, 76, 78])}
mini-batch 13: {'data': array([80, 82, 84])}
mini-batch 14: {'data': array([86, 88, 90])}
mini-batch 15: {'data': array([92, 94, 96])}


The functionality of the cell above is now pretty close to what the dataloader is supposed to do. However, there are still two remaining issues:
1. The last two samples of the dataset are not contained in any mini-batch. This is because the number of samples in the dataset is not dividable by the batch size, so there are a few left-over samples which are implicitly discarded. Ideally, an option would be prefered that allows you to decide how to handle these last samples.
2. The order of the mini-batches, as well as the fact which samples are grouped together, is always in increasing order. Ideally, there should be another option that allows you to randomize which samples are grouped together. The randomization could be easily implemented by randomly permuting the indices of the dataset before iterating over it, e.g. using `indices = np.random.permutation(len(dataset))`.

## 2. DataLoader Class Implementation
Now it is your turn to put everything together and implement the DataLoader as a proper class.
We provide you with a basic skeleton for this, which you can find in `class DataLoader` of `exercise_code/data/dataloader.py`. Open the file and have a look at the class. Note that the `__init__` method receives four arguments:
* **dataset** is the dataset that the dataloader should load.
* **batch_size** is the mini-batch size, i.e. the number of samples you want to load at a time.
* **shuffle** is a binary boolean (True or False) and defines whether the dataset should be randomly shuffled or not.
* **drop_last**: is binary and defines how to handle the last mini-batch in your dataset. Specifically, if the amount of samples in your dataset is not dividable by the mini-batch size, there will be some samples left over in the end. If `drop_last=True`, we simply discard those samples, otherwise we return them together as a smaller mini-batch.

<div class="alert alert-info">
    <h3>Task: Implement (1 Points)</h3>
    <p>Implement the <code>__len__(self)</code> method in <code>exercise_code/data/dataloader.py</code>. </p>
    <p><b>Hint:</b> Don't forget to think about drop_last! We will test for both modes.
</div>

In [20]:
from exercise_code.data.dataloader import DataLoader

dataloader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True
)

_ = test_dataloader_len(
    dataloader=dataloader
)


####### Testing Started #######

Testing __len__() with condition: drop_last=True. No. of test cases: 2
Test LenTestInt: passed!
Test LenTestCorrect: passed!
Method __len__() (using drop_last=True): passed! -->  Tests passed: 2/2

Testing __len__() with condition: drop_last=False. No. of test cases: 2
Test LenTestInt: passed!
Test LenTestCorrect: passed!
Method __len__() (using drop_last=False): passed! -->  Tests passed: 2/2

Testing __len__() with condition: drop_last=False; batch_size=1. No. of test cases: 2
Test LenTestInt: passed!
Test LenTestCorrect: passed!
Method __len__() (using drop_last=False; batch_size=1): passed! -->  Tests passed: 2/2


####### Testing Finished #######
Method __len__(): passed! -->  Tests passed: 6/6
Score: 100/100


<div class="alert alert-info">
    <h3>Task: Implement (3 Point)</h3>
    <p>Implement the <code>__iter__(self)</code> method in <code>exercise_code/data/dataloader.py</code>. </p>
    <p><b>Hint:</b> Make use of the code in '1. Iterating over a Dataset' when implementing your <code>__iter__()</code> method. We are again testing for both drop_last modes! 
</div>

In [22]:
from exercise_code.data.dataloader import DataLoader

dataloader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True,
)

_ = test_dataloader_iter(
    dataloader=dataloader
)


####### Testing Started #######

Testing __iter__() with condition: drop_last=True. No. of test cases: 8
Test IterTestIterable: passed!
Test IterTestItemType: passed!
Test IterTestBatchSize: passed!
Test IterTestNumBatches: passed!
Test IterTestValuesUnique: passed!
Test IterTestValueRange: passed!
Test IterTestShuffled: passed!
Test IterTestNonDeterministic: passed!
Method __iter__() (using drop_last=True): passed! -->  Tests passed: 8/8

Testing __iter__() with condition: drop_last=False. No. of test cases: 8
Test IterTestIterable: passed!
Test IterTestItemType: passed!
Test IterTestBatchSize: passed!
Test IterTestNumBatches: passed!
Test IterTestValuesUnique: passed!
Test IterTestValueRange: passed!
Test IterTestShuffled: passed!
Test IterTestNonDeterministic: passed!
Method __iter__() (using drop_last=False): passed! -->  Tests passed: 8/8


####### Testing Finished #######
Method __iter__(): passed! -->  Tests passed: 16/16
Score: 100/100


If you're done, run the cells below to check if your dataloader works as intended. You can change the value of drop_last to see the difference.

In [24]:
from exercise_code.data.dataloader import DataLoader

dataloader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False,    # Change here if you want to see the impact of drop last and check out the last batch
)
for batch in dataloader:
    print(batch)

{'data': array([ 4, 94, 90])}
{'data': array([84,  8, 48])}
{'data': array([38, 28,  6])}
{'data': array([76, 36, 30])}
{'data': array([12, 68, 60])}
{'data': array([20, 74, 56])}
{'data': array([34, 70, 64])}
{'data': array([100,  46,  86])}
{'data': array([32, 58, 52])}
{'data': array([22, 18,  2])}
{'data': array([62, 42, 16])}
{'data': array([72, 50, 82])}
{'data': array([26, 66, 98])}
{'data': array([80, 96, 14])}
{'data': array([78, 88, 40])}
{'data': array([10, 24, 92])}
{'data': array([44, 54])}


### Save your DataLoaders for Submission
Simply save your dataloaders using the following cell. This will save them as well as dataset from the first notebook to a pickle file `cifar_dataset_and_loader.p`.

In [26]:
from exercise_code.data.dataloader import DataLoader

dataloader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
)

dataset = load_pickle("cifar_dataset.p") # load dataset from the pickle file saved in notebook 1

save_pickle(
    data_dict={
        "dataset": dataset['dataset'],
        "cifar_mean": dataset['cifar_mean'],
        "cifar_std": dataset['cifar_std'],
        "dataloader": dataloader
    },
    file_name="cifar_dataset_and_loader.p"
)

<div class="alert alert-danger">
    <h3>Note</h3>
    <p>Note that <b>this is the ONLY file you need to submit</b>. Each time you make changes in either <code>dataset</code> or <code>dataloaders</code>, you need to <b>rerun the following code</b> to save your changes for submission.</p>
</div>

# Submission Instructions

Now, that you have completed the necessary parts in the notebook, you can go on and submit your files.

Go to the OpenOlat course:
https://lms.uni-kiel.de/url/RepositoryEntry/5692096707

Go under Exercises > Exercise 01

!!! You only see this if you are enrolled into one of the groups !!!

Submit your exercise there.


In [30]:
from exercise_code.submit import submit_exercise

submit_exercise('../output/exercise_02_group_11.zip')

relevant folders: ['exercise_code', 'models']
notebooks files: ['exercise_02_1_cifar10.ipynb', 'exercise_02_2_dataloader.ipynb']
Adding folder exercise_code
Adding folder models
Adding notebook exercise_02_1_cifar10.ipynb
Adding notebook exercise_02_2_dataloader.ipynb
Zipping successful! Zip is stored under: C:\Users\Yanne\Desktop\Foundations of deep learning\GIT\i2dl_exercises\output\exercise_02_group_11.zip


# Submission Goals

For this exercise we only test your implementations which are tested throughout both notebooks. Here is a list of test cases that will be evaluated on the server using your `ImageFolderDataset` as well as `DataLoader` classes. Here is an overview split among our two notebooks:

- Goal for **notebook 1**: Implement an ImageFolderDataset with transforms for rescaling and normalizing.
    - To implement: 
        1. `exercise_code/data/image_folder_dataset.py`: `ImageFolderDataset` -  `__len__()`, `__getitem()__`
        2. `exercise_code/data/image_folder_dataset.py`: `RescaleTransform`
        3. `exercise_code/data/image_folder_dataset.py`: `compute_image_mean_and_std()`
    - Test cases:
      1. Does `__len__()` of `ImageFolderDataset` return the correct data type?
      2. Does `__len__()` of `ImageFolderDataset` return the correct value?
      3. Does `__getitem()__` of `ImageFolderDataset` return the correct data type?
      4. Does `__getitem()__` of `ImageFolderDataset` load images as numpy arrays with correct shape?
      5. Do values after rescaling with `RescaleTransform` have the correct minimum?
      6. Do values after rescaling with `RescaleTransform` have the correct maximum?
      7. Does `compute_image_mean_and_std()` compute the correct mean?
      8. Does `compute_image_mean_and_std()` compute the correct std?


- Goal for **notebook 2**: Implement a DataLoader that loads mini-batches from a given dataset and supports batch_size, shuffle, and drop_last args.
    - Test cases:
      1. Does `__len__()` return the correct data type?
      2. Does `__len__()` return the correct value?
      3. Does `__iter__()` work at all, i.e. is it possible to iterate over the dataloader?
      4. Does `__iter__()` load the correct data type?
      5. Does `__iter__()` load data with correct batch size?
      6. Does `__iter__()` load the correct number of batches?
      7. Does `__iter__()` load every sample only once?
      8. Does `__iter__()` load the smallest and largest sample from the dataset?
      9. Does `__iter__()` shuffle the data correctly (if necessary)?
      10. Does `__iter__()` return non-deterministic values when shuffling?

# Key Takeaways
1. In machine learning, we often need to load data in **mini-batches**, which are small subsets of the training dataset. How many samples to load per mini-batch is called the **batch size**.
2. In addition to the Dataset class, we use a **DataLoader** class that takes care of mini-batch construction, data shuffling, and more.
3. The dataloader is iterable and only loads those samples of the dataset that are needed for the current mini-batch. This can lead to bottlenecks later if you are unable to provide enough batches in time for your upcoming pipeline. This is especially true when loading from HDDs as the slow reading time can be a bottleneck in your complete pipeline later.
4. The dataloader task can easily by distributed amongst multiple processes as well as pre-fetched. When we switch to PyTorch later we can directly use our dataset classes and replace our current Dataloader with theirs :).